In [1]:
import os, nest_asyncio
from IPython.display import Markdown, display
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext, get_response_synthesizer, VectorStoreIndex, SimpleDirectoryReader, PromptTemplate
os.chdir('/Users/1zuu/Library/Mobile Documents/com~apple~CloudDocs/ML DEV/LLM RESEARCH 2024/LLM in Production/03-diy-llm-chat-bot-e2e-mongodb')
nest_asyncio.apply() 

In [2]:
from util.llm_services import *

Using Groq LLM ....
Using HuggingFace Embedding ....


/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
documents = SimpleDirectoryReader(path_config.kb_documents_dir).load_data()
documents

[Document(id_='de860d68-c405-445f-afc6-f081d555bd2a', embedding=None, metadata={'page_label': '1', 'file_name': 'HNB.pdf', 'file_path': '/Users/1zuu/Library/Mobile Documents/com~apple~CloudDocs/ML DEV/LLM RESEARCH 2024/LLM in Production/03-diy-llm-chat-bot-e2e-mongodb/db/kb/HNB.pdf', 'file_type': 'application/pdf', 'file_size': 2653957, 'creation_date': '2024-06-04', 'last_modified_date': '2024-05-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='FIXED DEPOSIT SERVICES\nCIB/CMB User Information ManualCorporate Internet Banking', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='7404c32f-1620-4e52-bbc5-44e666b0ccad', embedding=None, metadata

In [4]:
nodes = Settings.node_parser.get_nodes_from_documents(documents)
print(f"Total nodes: {len(nodes)}")

Total nodes: 16


In [5]:
collection_name = "embeddings_bank"
vector_store = MongoDBAtlasVectorSearch(
                                    db_name=db_config.mongo_db,
                                    client=mongo_client, 
                                    collection_name=collection_name
                                    )
if not collection_name in mongo_client[db_config.mongo_db].list_collection_names():
    print("Building Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store,
                                                    )
    index = VectorStoreIndex(
                            nodes, 
                            storage_context=storage_context
                            )
    
else:
    print("Loading Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store
                                                    )
    index = VectorStoreIndex.from_vector_store(
                                                vector_store, 
                                                storage_context=storage_context
                                                )

retriever = VectorIndexRetriever(
                                index=index,
                                similarity_top_k=3,
                                )

response_synthesizer = get_response_synthesizer()
query_engine = RetrieverQueryEngine(
                                    retriever=retriever,
                                    response_synthesizer=response_synthesizer,
                                    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.0)],
                                    )

Building Finance Index ...


In [6]:
question = "What are the Eligibility criteria to open a Fixed Deposit?"

In [8]:
response = query_engine.query(question)
display(Markdown(str(response)))

Empty Response

### Reproduce the Query Engine with new prompt

In [8]:
collection_name = "embeddings_bank"
vector_store = MongoDBAtlasVectorSearch(
                                    db_name=db_config.mongo_db,
                                    client=mongo_client, 
                                    collection_name=collection_name
                                    )
if not collection_name in mongo_client[db_config.mongo_db].list_collection_names():
    print("Building Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store,
                                                    )
    index = VectorStoreIndex(
                            nodes, 
                            storage_context=storage_context
                            )
    
else:
    print("Loading Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store
                                                    )
    index = VectorStoreIndex.from_vector_store(
                                                vector_store, 
                                                storage_context=storage_context
                                                )

retriever = VectorIndexRetriever(
                                index=index,
                                similarity_top_k=3,
                                )

response_synthesizer = get_response_synthesizer()
query_engine = RetrieverQueryEngine(
                                    retriever=retriever,
                                    response_synthesizer=response_synthesizer,
                                    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
                                    )


text_chat_template = """
Context Information is below.
---------------------
{context_str}
---------------------

Chat History and Query is below.
---------------------
{query_str}
---------------------

Given the Context Information, Chat History and not prior knowledge, answer the query.

ASSISTANT: 
"""

text_chat_pmt_template = PromptTemplate(text_chat_template)   

query_engine.update_prompts(
                            {"response_synthesizer:text_qa_template": text_chat_pmt_template}
)

Loading Finance Index ...


In [9]:
prompts_dict = query_engine.get_prompts()
print(prompts_dict['response_synthesizer:text_qa_template'])

metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['context_str', 'query_str'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='\nContext Information is below.\n---------------------\n{context_str}\n---------------------\n\nChat History and Query is below.\n---------------------\n{query_str}\n---------------------\n\nGiven the Context Information, Chat History and not prior knowledge, answer the query.\n\nASSISTANT: \n'


In [10]:
query_engine.query("""USER : Hi !\nASSISTANT : Hello! How can I help you today?\nUSER : What are the Eligibility criteria to open a Fixed Deposit?\n""")

Response(response='Empty Response', source_nodes=[], metadata=None)

In [12]:
query_engine.retrieve("""USER : Hi !\nASSISTANT : Hello! How can I help you today?\nUSER : What are the Eligibility criteria to open a Fixed Deposit?\n""")

[]